In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary

In [ ]:
num_features = 50
num_outputs = 3

### Manual


In [ ]:
# Build a neural network model using nn.Sequential
model = nn.Sequential(
    # First linear layer: transforms input features to 24 features
    nn.Linear(num_features, 24),
    # Activation function: applies ReLU non-linearity
    nn.ReLU(),
    # Second linear layer: reduces dimensions to 12 features
    nn.Linear(24, 12),
    # Activation function: applies ReLU non-linearity
    nn.ReLU(),
    # Third linear layer: reduces dimensions to 6 features
    nn.Linear(12, 6),
    # Activation function: applies ReLU non-linearity
    nn.ReLU(),
    # Output layer: maps 6 features to the desired number of output classes/values
    nn.Linear(6, num_outputs),
)

In [ ]:
batch_size = 100
input_size = (batch_size, num_features)
summary(model, input_size=input_size)

In [ ]:
batch_size = 10
X = torch.randn(batch_size, num_features)
Y = model(X)
print(X.shape, Y.shape)

### Class (recommended)


In [ ]:
class MyMLP(nn.Module):
    def __init__(self, num_features, num_outputs):
        # This line calls the constructor (initializer method) of the parent class, which in this case is nn.Module (from PyTorch).
        # When you create a custom neural network by subclassing nn.Module, you must ensure that all the necessary initialization in nn.Module is done properly.
        # The super() function allows your class (MyMLP) to inherit and use methods and properties from its parent (nn.Module).
        # Without this call, things like parameter registration and model functionality in PyTorch may not work correctly.
        super(MyMLP, self).__init__()

        # First fully connected layer: input size = num_features, output size = 24
        self.fc1 = nn.Linear(num_features, 24)

        # Second fully connected layer: input size = 24, output size = 12
        self.fc2 = nn.Linear(24, 12)

        # Third fully connected layer: input size = 12, output size = 6
        self.fc3 = nn.Linear(12, 6)

        # Final fully connected layer: input size = 6, output size = num_outputs
        self.fc4 = nn.Linear(6, num_outputs)

        # ReLU activation function for introducing non-linearity
        self.relu = nn.ReLU()

    def forward(self, x):
        # Pass input through first layer, then apply ReLU
        x = self.fc1(x)
        x = self.relu(x)

        # Pass through second layer and apply ReLU
        x = self.fc2(x)
        x = self.relu(x)

        # Pass through third layer and apply ReLU
        x = self.fc3(x)
        x = self.relu(x)

        # Pass through output layer (no activation)
        x = self.fc4(x)

        return x  # Return the output (raw predictions)


In [ ]:
# A more concised version
class MyMLP_V2(nn.Module):
    def __init__(self, num_features, num_outputs):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 24)
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, 6)
        self.fc4 = nn.Linear(6, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
batch_size = 100
input_size = (batch_size, num_features)
summary(model, input_size=input_size)

In [ ]:
# Example usage
model = MyMLP(num_features, num_outputs)
batch_size = 10
X = torch.randn(batch_size, num_features)
Y = model(X)
print(X.shape, Y.shape)

### More complicated example


In [ ]:
class MyComplicatedMLP(nn.Module):
    def __init__(self, input_dim=50, hidden_dim=64, output_dim=1):
        super(MyComplicatedMLP, self).__init__()

        # Linear layers for splitting
        self.fc_first = nn.Linear(25, 32)
        self.fc_latter = nn.Linear(25, 32)

        # Activation functions
        self.act_first = nn.ReLU()
        self.act_latter = nn.Sigmoid()

        # Combine and further processing
        self.fc_combined = nn.Linear(64, hidden_dim)
        self.act_combined = nn.ReLU()
        self.fc_output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Split input: x[:, :25] and x[:, 25:]
        x_first = self.act_first(self.fc_first(x[:, :25]))
        x_latter = self.act_latter(self.fc_latter(x[:, 25:]))

        # Concatenate
        x_combined = torch.cat([x_first, x_latter], dim=1)
        x = self.act_combined(self.fc_combined(x_combined))
        out = self.fc_output(x)
        return out

In [ ]:
model = MyComplicatedMLP()
input_size = (100, 50)
summary(model, input_size=input_size)

In [ ]:
# Example usage
model = MyComplicatedMLP()
sample_input = torch.randn(10, 50)  # Batch size of 10
output = model(sample_input)
print(output.shape)